In [2]:
import landsat_sample_functions as lf

In [3]:
import glob
import os
import re
import datetime
dexpr = re.compile("\d{4}[-/]\d{2}[-/]\d{2}")

In [4]:
rgbf = glob.glob('../data/landast_archibe/RGB_subset_*')
rthermrad = glob.glob('../data/landast_archibe/landsat_thermrad_*')
rflect = glob.glob('../data/landast_archibe/landsat_reflectances_*')
latlon = glob.glob('../data/landast_archibe/landsat_latlon_*')


In [5]:
def unpack(filename):
    _, fullname = os.path.split(filename)
    name, ext = os.path.splitext(fullname)
    date = dexpr.findall(name)[0]
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    if ext == '.txt':
        data = lf.landsat_read(filename)
    return (dt, data)

In [7]:
data = dict()
for files, key in [(rthermrad, "thermrad"), (rflect, "reflectances"), (latlon,"latlon")]:
    data[key] = [unpack(fn) for fn in sorted(files)]


In [ ]:
import xarray as xr
import numpy as np

In [ ]:
dv = [np.stack(zip(*data[k])[-1]) for k in ['thermrad', 'reflectances']]

In [ ]:
landsat = np.concatenate( [dv[1], dv[0][...,np.newaxis]], axis=-1)

In [ ]:
landsat.shape

In [ ]:
layers  = ['NIR', 'Red', 'Green', 'Blue', 'UV', "Thermal Radiances"]

In [ ]:
times, latlon = zip(*data['latlon'])

In [ ]:
locs = np.stack(latlon)

In [ ]:
lats = locs[...,0]
lons = locs[...,1]

In [ ]:
lats.shape

In [ ]:
import pandas as pd
coords = {'lon':(['time','y', 'x'], lons),
          'lat':(['time','y', 'x'], lats) ,
          'time':pd.to_datetime([t.date() for t in times])}

In [ ]:
data = dict()
for l, d in zip(layers, landsat.transpose([-1,0, 1,2])):
    print(l, d.shape)
    data[l] = (['time','y', 'x'], d[:, ::-1, :])

In [ ]:
ds = xr.Dataset(data, coords=coords)

In [ ]:
%matplotlib inline
_ = ds['Red'].mean('time').plot(cmap='Reds')

In [ ]:
ds.to_netcdf('landsat_hires.nc')